In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import skimage
from pathlib import Path
from scipy.io import loadmat
import pickle
from copy import deepcopy

In [ ]:
from bayesee.perception import *
from bayesee.validation import *
from bayesee.visualization import *
from bayesee.generation import *
from bayesee.evaluation import *
from bayesee.iteration import *
from bayesee.partition import *

In [ ]:
%load_ext autoreload
%autoreload 2
plt.style.use('bayesee.academic')

In [ ]:
repo_path = Path.cwd().parents[0]
print(repo_path)

In [ ]:
subject = "AZ"
file_name = repo_path / f"data/discrimination/location/data_{subject}.pickle"

with open(file_name, "rb") as f:
    stimulus, response = pickle.load(f)

metadata = stimulus["metadata"]
print(metadata.keys())

In [ ]:
stimulus_iso = deepcopy(stimulus)
response_iso = deepcopy(response)

stimulus_iso["df"].rename(columns={"location": "presence"}, inplace=True)
response_iso["df"].rename(
    columns={"response_location": "response_presence"}, inplace=True
)

stimulus_iso["df"]["presence"] -= 1
response_iso["df"]["response_presence"] -= 1

In [ ]:
array_dprime, array_criterion = compute_dprime_criterion_w_discrete_response(
    stimulus_iso, response_iso
)

array_PC_max = compute_PC_max_w_dprime(array_dprime)

In [ ]:
fig, ax = plt.subplots()
amplitude_unique, index_unique = np.unique(
    stimulus["df"]["amplitude"], return_index=True
)

ax.scatter(
    amplitude_unique,
    array_PC_max[index_unique],
    s=250,
    label="PC$_{max}$",
)
ax.scatter(
    amplitude_unique,
    array_criterion[index_unique],
    s=250,
    label="$\gamma$",
)

ax.legend(loc="best")

ax.set(xlabel="Displacement amplitude (arc min)", ylabel="Value")

In [ ]:
x0 = [3.5, 0]

results = minimize(
    negative_loglikelihood_w_parameter,
    x0,
    args=(stimulus, response, unit_likelihood_gaussian),
)

results

In [ ]:
sigma, gamma = results.x

In [ ]:
fig, ax = plt.subplots()
amplitude_unique = np.unique(stimulus["df"]["amplitude"])

ax.scatter(
    amplitude_unique,
    array_PC_max[index_unique],
    s=250,
    label="PC$_{max}$",
)
ax.scatter(
    amplitude_unique,
    array_criterion[index_unique],
    s=250,
    label="$\gamma$",
)

amplitude_full = np.linspace(0, amplitude_unique.max() * 1.1, 50)

ax.plot(
    amplitude_full,
    norm.cdf(amplitude_full / sigma),
    label="Gaussian",
)

ax.legend(loc="best")
ax.set(
    xlabel="Displacement amplitude (arc min)",
    ylabel="Value",
)
ax.set_xlim(left=0)

In [ ]:
x0 = [7, 0]

results = minimize(
    negative_loglikelihood_w_parameter,
    x0,
    args=(stimulus, response, unit_likelihood_uniform),
    bounds=((0, np.inf), (-3, 3)),
    method="Powell",
)

results

In [ ]:
rho, gamma = results.x

In [ ]:
fig, ax = plt.subplots()
amplitude_unique = np.unique(stimulus["df"]["amplitude"])

ax.scatter(
    amplitude_unique,
    array_PC_max[index_unique],
    s=250,
    label="PC$_{max}$",
)
ax.scatter(
    amplitude_unique,
    array_criterion[index_unique],
    s=250,
    label="$\gamma$",
)

amplitude_full = np.linspace(0, amplitude_unique.max() * 1.1, 50)

ax.plot(
    amplitude_full,
    (
        np.arccos(-amplitude_full / rho)
        + amplitude_full / (rho**2) * np.sqrt(rho**2 - amplitude_full**2)
    )
    / np.pi,
    label="Uniform",
)

ax.legend(loc="best")
ax.set(
    xlabel="Displacement amplitude (arc min)",
    ylabel="Value",
)
ax.set_xlim(left=0)

In [ ]:
fig, ax = plt.subplots()
ax2 = ax.twinx()

array_subject = ["AZ", "HS"]
array_color = ["#377eb8", "#ff7f00"]

for index_subject, subject in enumerate(array_subject):
    file_name = repo_path / f"data/discrimination/location/data_{subject}.pickle"

    with open(file_name, "rb") as f:
        stimulus, response = pickle.load(f)

    metadata = stimulus["metadata"]
    stimulus_iso = deepcopy(stimulus)
    response_iso = deepcopy(response)

    stimulus_iso["df"].rename(columns={"location": "presence"}, inplace=True)
    response_iso["df"].rename(
        columns={"response_location": "response_presence"}, inplace=True
    )

    stimulus_iso["df"]["presence"] -= 1
    response_iso["df"]["response_presence"] -= 1
    array_dprime, array_criterion = compute_dprime_criterion_w_discrete_response(
        stimulus_iso, response_iso
    )

    array_PC_max = compute_PC_max_w_dprime(array_dprime)

    amplitude_unique = np.unique(stimulus["df"]["amplitude"])

    ax.scatter(
        amplitude_unique,
        array_PC_max[index_unique],
        s=250,
        label=subject,
        c=array_color[index_subject],
    )
    ax2.scatter(
        amplitude_unique,
        array_criterion[index_unique],
        s=250,
        lw=5,
        edgecolors=array_color[index_subject],
        facecolors="none",
    )

    amplitude_full = np.linspace(0, amplitude_unique.max() * 1.1, 50)

    x0 = [3.5, 0]

    results = minimize(
        negative_loglikelihood_w_parameter,
        x0,
        args=(stimulus, response, unit_likelihood_gaussian),
    )

    sigma, gamma = results.x

    ax.plot(
        amplitude_full,
        norm.cdf(amplitude_full / sigma),
        "-",
        c=array_color[index_subject],
        lw=5,
        alpha=0.35,
    )

    x0 = [7, 0]

    results = minimize(
        negative_loglikelihood_w_parameter,
        x0,
        args=(stimulus, response, unit_likelihood_uniform),
        bounds=((0, np.inf), (-3, 3)),
        method="Powell",
    )

    rho, gamma = results.x

    ax.plot(
        amplitude_full,
        (
            np.arccos(-amplitude_full / rho)
            + amplitude_full / (rho**2) * np.sqrt(rho**2 - amplitude_full**2)
        )
        / np.pi,
        "--",
        c=array_color[index_subject],
        lw=5,
        alpha=0.35,
    )

ax.plot(0, 0.5, "-", c="k", label="Gaussian", lw=5, zorder=-1)
ax.plot(0, 0.5, "--", c="k", label="Uniform", lw=5, zorder=-1)
ax2.scatter(
    amplitude_unique[0],
    array_criterion[index_unique][0],
    s=250,
    facecolors="none",
    linewidths=5,
    edgecolors="k",
    label="$\gamma$",
    zorder=-1,
)

ax.legend(loc="upper left")
ax2.legend(loc="lower right")

ax.set_ylim([0.3, 1.1])

labels = [f"{item:.1f}" for item in ax.get_yticks()]
for i in [0, 1, -1]:
    labels[i] = ""
    ax.yaxis.get_major_ticks()[i].set_visible(False)

ax.set(
    xlim=[0, 7],
    xlabel="Displacement amplitude (arc min)",
    ylabel="PC$_{max}$",
    yticklabels=labels,
)

ax2.set_ylim([-1.5, 6])

labels = [f"{item:.1f}" for item in ax2.get_yticks()]
for i in [4, 5, 6, 7, 8]:
    labels[i] = ""
    ax2.yaxis.get_major_ticks()[i].set_visible(False)

ax2.set(yticklabels=labels)

fig.text(
    1.01,
    0.25,
    "Criterion",
    va="center",
    rotation=-90,
)